In [2]:
import os
path = os.getcwd()

if path[1:5] == "Home":
    %run /Home/siv30/wad005/master/GRB/src/read_matlab_file.ipynb
    %run /Home/siv30/wad005/master/GRB/src/convert_erg.ipynb
    %run /Home/siv30/wad005/master/GRB/src/safety_time.ipynb
else:
    %run /Users/andreas/phys/master/GRB/src/read_matlab_file.ipynb
    %run /Users/andreas/phys/master/GRB/src/convert_erg.ipynb
    %run /Users/andreas/phys/master/GRB/src/safety_time.ipynb

All MAC paths are set!
read_matlab_file.ipynb loaded successfully!
All MAC paths are set!
read_matlab_file.ipynb loaded successfully!
convert_erg.ipynb loaded successfully!
All MAC paths are set!
read_matlab_file.ipynb loaded successfully!
All MAC paths are set!
read_matlab_file.ipynb loaded successfully!
convert_erg.ipynb loaded successfully!
safety_time.ipynb loaded successfully!


# implementation blueprint
        
    GENERAL IMPLEMENTATION STEPS
    ----------------------------    
    
    
    DATA HANDLING
    -------------
    1. Subtract absolute time correction from T0 and tus array
    2. importing calibration coef and coverting to keV
    3. creating dataframe containing data
    4. sorting by adr and then tus
    5. estimating dt between i and i-1 (after sorting by adr and tus)
    6. calculating safety time based on keV and dt
    7. flagging events that have dt <= st AND typ > 1 events.

    LC PRODUCTION
    -----------------
    1. creating LC containing complete photon list. DONE
    2. filtering photon list based on relevant energy interval. HED [350-1500 kev], LED [80-350 kev]. DONE
    3. remove flagged events from the filtered photon list
    4. calculate background for the relevant energy bands 
    5. plotting LC where background is subtracted
    
    
    final step. plot LC of the flagged events only. The ratio between the flagged events / number of counts in each bin is a measure of the dead time. (deadtime relative to binsize) 
    
    
    

# complete df

In [1]:
def complete_df(grb_id=None, remove_flag=False, lag=None, flag_cut="HED"):
    '''
    TODO: Use np.float32 on cal_abc array
    
    
    
    input: grb_id <str>: Name of gammaray burst id of type GRB200415
    remove_flag <bool>: standard to False. If the returned df will contain any flagged events
    lag <float> [ms] : lag is added to the tus array (corrolated with another spacecraft). 
    
    notes: Valleys or typ=3 events will be removed from the df anyway
    Long GRB's have to be concatenated
    
    If flag=True the returned df will not contain any st flagged events
       
    else: keeping flagged events
    
    return: <pd.dataframe> df sorted by tus in increasing order
    '''
    
    
    if len(trig_dict_HED[grb_id])>1:
        print("Creating df for long GRB")
        
        #handling the long GRBs
        
        #Need dau & det arrays for the energy calibration
        # ---> Making them flow through with tus and erg.
        
        #need also typ and adr array
        tus_cut, erg_cut, typ_cut,adr_cut, dau_cut, det_cut, T0 = cut_cat(grb_id=grb_id, flag=flag_cut) #cutcat gets the concatinated arrays from grb_id
        
        keV, cal_abc = convert_erg(erg_cut,dau_cut,det_cut,T0)
        print("erg converted to keV")
        
        df = create_df(tus_cut, erg_cut, typ_cut, adr_cut, dau_cut, det_cut, keV, cal_abc)
        # same handling of the df from here 
        #----------------------------------------------------
       
    
    else:
        print("Creating df for short GRB")
        tus, erg, typ, adr, dau, det, T0 = read_matlab_file_with_BGO_data(trig_dict_HED[grb_id][0])
        keV,cal_abc = convert_erg(erg, dau, det, T0)
        print("erg converted to keV")
        df = create_df(tus, erg, typ, adr, dau,det ,keV, cal_abc)
        
        # same handling of the df from here 
        #----------------------------------------------------
        
    df = df.sort_values(by=['adr','tus']) #important to sort first by the adr then tus --> Handling the triggers in each detector seperate
    print("df sorted by adr and tus")
    #Creating columns for st, dt and flag
    st_list = get_st(df)
    df["st"] = st_list

    dt_list = get_dt(df)
    df["dt"] = dt_list

    flag_arr = get_flag(df) #should also flag fast events!
    df["flag"] = flag_arr

    print("safety time events flagged. " + "Found " + str(sum(df["flag"].to_numpy())) + " flags")

    if remove_flag==False:
        print("returning df containing flagged events")
        return df,T0
    else:
        flag_label = []
        for flag_idx,row in df.iterrows():
            if row["flag"]==True:
                flag_label.append(flag_idx)

        if len(flag_label) == 0:
            return print("No flags found. Check for error")

        else:
            print("Removed " + str(len(flag_label)) + " flags from df")
            df.drop(labels=flag_label,inplace=True)
            print("df sorted by tus")
            df.sort_values(by=['tus'],inplace=True)
            
            if lag != None:
                df["tus"] = df["tus"].to_numpy() + lag*1e3 #converting lag in ms into us
                return df, T0
            else:
                return df, T0

# filtering energy interval and removing flagged events

In [ ]:
# df is soreted by adr and then tus

def filter_by_energy_interval(df,lower_bound,upper_bound):
    """lower and upper bound given in keV
    
    Note: have to remove the flagged ST events because one can not trust the energy for those flagged events

    return: tus: photon trigger list filtered by energy interval
            kev: array of keV's 
            flagged: array containing the flaged tus in the energy interval"""
    
    tus_arr = df["tus"].to_numpy()
    kev_arr = df["keV"].to_numpy()
    flag_arr = df["flag"].to_numpy()
    
    
    tus_temp, kev_temp, flagged_temp = [],[],[]
    print("selecting kev interval. Removing flagged events.")
    for i,kev in enumerate(kev_arr):
        if lower_bound <= kev <= upper_bound:
            if flag_arr[i] == False:
                kev_temp.append(kev)
                tus_temp.append(tus_arr[i])
            else:
                flagged_temp.append(tus_arr[i])

    print("found {} out of {} flagged triggers on the given keV interval".format(len(flagged_temp), sum(df["flag"])))
    return np.array(tus_temp), np.array(kev_temp), np.array(flagged_temp)
    

In [ ]:
def estimate_dead_time(df=None,binsize=None):
    '''defined as the the ratio between the counts in the filtered (or the normal array) / removed bins
    generating the ratio between the normal and filtered hists
    
    
    NOTE: binning based on the TUS_FILT array.
    
    estimate the ratio between the number of flagged events vs filtered counts in each bin. 
    how to estimate dead time? 
    
    return the ratio pr bin as an array'''
    
    tus = df["tus"].to_numpy()
    flag = df["flag"].to_numpy()
    
    mask_noflag, mask_flag = np.where(flag == False)[0], np.where(flag == True)[0]

    tus_noflag = tus[mask_noflag]
    tus_flag = tus[mask_flag]
    
    hist_noflag, edges_noflag = np.histogram(tus_noflag/1e3,bins=np.arange(min(tus)/1e3,max(tus)/1e3 + binsize, binsize))
    hist_flag, edges_flag = np.histogram(tus_flag/1e3,bins=np.arange(min(tus)/1e3,max(tus)/1e3 + binsize, binsize))
    
    fraq = []
    
    for i,bin_count in enumerate(hist_noflag):
        if bin_count == 0:
            fraq.append(0)
        else:
            fraq.append(hist_flag[i]/bin_count)
    
    return fraq,edges_noflag
    

In [ ]:
print("LC_calibration loaded successfully!")